In [1]:
%load_ext aiida
%aiida

Loaded AiiDA DB environment - profile name: default.

In [2]:
import ipywidgets as ipw
from aiida.orm import StructureData
from datetime import datetime
import jinja2

In [3]:
qb = QueryBuilder()
qb.append(StructureData, filters={
    "extras": {"has_key": "surfaces"}}
)

QueryBuilder(path=[{'entity_type': 'data.core.structure.StructureData.', 'orm_base': 'node', 'tag': 'StructureData_1', 'joining_keyword': None, 'joining_value': None, 'edge_tag': None, 'outerjoin': False}], filters={'StructureData_1': {'node_type': {'like': 'data.core.structure.%'}, 'extras': {'has_key': 'surfaces'}}}, project={'StructureData_1': []}, order_by=[], limit=None, offset=None, distinct=False)

In [4]:
#        filters={
#            "attributes.process_label": {"in": workchain_list},
#            "or": [
#                {"extras": {"!has_key": "preprocess_version"}},
#                {"extras.preprocess_version": {"<": PREPROCESS_VERSION}},
#            ],
#        },
#    )
qb.order_by({StructureData: {"mtime": "desc"}})

QueryBuilder(path=[{'entity_type': 'data.core.structure.StructureData.', 'orm_base': 'node', 'tag': 'StructureData_1', 'joining_keyword': None, 'joining_value': None, 'edge_tag': None, 'outerjoin': False}], filters={'StructureData_1': {'node_type': {'like': 'data.core.structure.%'}, 'extras': {'has_key': 'surfaces'}}}, project={'StructureData_1': []}, order_by=[{'StructureData_1': [{'mtime': {'order': 'desc'}}]}], limit=None, offset=None, distinct=False)

In [5]:
table = ipw.HTML()
display(table)

HTML(value='')

In [17]:
def thunmnail_raw(nrows=1,thumbnail=None,uuid=None,pk=None,description=''):
    # image with a link to structure export
    
    html = (
        '<td rowspan=%s><a target="_blank" href="./export_structure.ipynb?uuid=%s">'
        %(str(nrows),uuid)
    )
    html += (
        '<img width="100px" src="data:image/png;base64,%s" title="PK%d: %s">'
        % (thumbnail, pk, description)
    )
    html += "</a></td>"
    return html
def link_to_viewer(description='',uuid=''):
    return '<li><a target="_blank" href="./export_structure.ipynb?uuid=%s"> %s </a></li>'% (uuid,description)
def nodes_to_dict(nodes):
        dict_workchains = {}
        for node in nodes:
            if node.label in dict_workchains:
                dict_workchains[node.label].append(node)
            else:
                dict_workchains[node.label] = [node]

        return dict_workchains
            

In [18]:
# date type description thumb
data = []
for node_tuple in qb.iterall():
    node = node_tuple[0]
    workflows_uuids = []
    for key in node.extras['surfaces']:
        for uuid in node.extras['surfaces'][key]: # e.g. list of all geo_opt
            workflows_uuids.append(load_node(uuid)) 
    dict_uuids = nodes_to_dict(workflows_uuids)
    nrows = sum([len(dict_uuids[k]) for k in dict_uuids])
    entry = {
        "nrows" : nrows,
        "mtime": node.mtime.strftime("%d/%m/%y"),
        "workflows": dict_uuids,
        "thumbnail":thunmnail_raw(nrows=nrows,thumbnail=node.extras['thumbnail'],uuid=node.uuid,pk=node.pk,description='')
    }
    
    data.append(entry)

In [19]:
#template = jinja2.Template(html_template)
#table.value=template.render(data=data)
display(table)

HTML(value='<style>#aiida_results td,th {padding: 2px}</style>\n<table border=1 id="aiida_results" style="marg…

In [ ]:
table.value="""
<style>#aiida_results td,th {padding: 2px}</style>
<table border=1 id="aiida_results" style="margin:0px">
<thead>
  <tr>
    <th >Date</th>
    <th >WOrK</th>
    <th >Desc</th>
    <th >TH</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td rowspan="5">Feb</td>
    <td rowspan="3">Ge</td>
    <td rowspan="3">1<br>2<br>3</td> r1
    <td rowspan="5">tum</td>
  </tr>
  
   <tr>
  </tr> r2 
  

  <tr> r3
  </tr>
  
  <tr>
    <td  rowspan="2">St</td> r4
    <td  rowspan="2">1<br>2</td>
  </tr>
  
  <tr>
  </tr> r5
  
  <tr>
    <td  rowspan="4">Set</td> 
    <td  rowspan="2">St</td>
    <td  rowspan="2">1<br> 2</td>
    <td  rowspan="4">tum</td>
  </tr>
  <tr>
  </tr>
  <tr>
    <td >Ge</td>
    <td >10</td>
  </tr>
  <tr>
    <td >AF</td>
    <td >1</td>
  </tr>
  <tr>
    <td  rowspan="2">Nov</td>
    <td  rowspan="2">st</td>
    <td  rowspan="2">10<br>20</td>
    <td  rowspan="2">tum</td>
  </tr>
  <tr>
  </tr>
</tbody>
</table>
"""

In [30]:
html = """<style>#aiida_results td,th {padding: 2px}</style>
<table border=1 id="aiida_results" style="margin:0px">
<thead>
  <tr>
    <th >Date last</th>
    <th >Calc. Type</th>
    <th >Description</th>
    <th >Thumbnail</th>
  </tr>
</thead>
<tbody>"""
for entry in data:
    nrows1 = entry['nrows']
    nrows_done = 0
    html += '<tr>'
    html += '<td rowspan=%s> %s  </td>' %(str(nrows1),entry['mtime'])
    for workflow in entry['workflows']:
        nrowsw = len(entry['workflows'][workflow])
        html += '<td rowspan=%s>  %s </td>' %(str(nrowsw),workflow)
        html+= '<td><ul>'
        for node in entry['workflows'][workflow]:
            print(node.description,node.pk)
            html += link_to_viewer(description='PK-'+str(node.pk)+' '+node.description,uuid=node.uuid)
        html += '</td></ul>'
        if nrows_done == 0:
            html += entry['thumbnail']
            nrows_done = 1
        for tr_empty in range(1,nrowsw):
            html += '<tr></tr>'
    html += '</tr>'
html += '</tbody></table>'

    
    

test description 73
a new opt 109
test description 90
test description 56
test description 39
test description 22
test description 5


In [ ]:
a={'a':[1,2,3],'b':[10,20]}
sum([len(a[k]) for k in a])

In [ ]:
entry = data[0]

In [ ]:
sum([len(entry['workflows'][k]) for k in entry['workflows']])

In [ ]:
for workflow in entry['workflows']:
    nrowsw = len(entry['workflows'][workflow])
    print(nrowsw)

In [ ]:
for i in range(1,2):
    print(i)

In [15]:
table.value = html

In [27]:
node = load_node('149cec55-1690-4358-9460-d4f39648a0c5')

In [28]:
link_to_viewer(description='PK-'+str(node.pk)+' '+node.description,uuid=node.uuid)

'<li><a target="_blank" href="./export_structure.ipynb?uuid=149cec55-1690-4358-9460-d4f39648a0c5"> PK-73 test description </a></li>'

In [24]:
node.description

''

In [26]:
node.extras

{'test': 'something',
 'formula': 'Si8',
 'surfaces': {'geo_opt': ['149cec55-1690-4358-9460-d4f39648a0c5',
   'eb9c6a76-3f8d-43bb-9639-f70ee5fa4785']},
 'thumbnail': 'iVBORw0KGgoAAAANSUhEUgAAAIkAAACJCAYAAAAYJBvJAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy88F64QAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAn5ElEQVR4nO2dZ3gUVdvH/2d2dmdnd5MQeif00Kt0CJ2HooAUlV6VjoICFuQBUZqAEIpKE6RIL4IgUkPvoUgCARI6BBJStszszsx5PwR4AibZXngvf9e1X2bOnHPvzr0z59znLoRSin/5l5xg/C3AvwQ+/yrJv9iHUur1D4AUADTT51Cmc4e8edwXY/w/HTvlxXHiizkJIYRSSonXB/oXj5H5nv37uvkXu/hKSQ77aJx/8Rwv75lPXjfeghCiBlAJQC0tp66mYdlchCE8pdQqy4rRZBGvAjgHIJpSavSvtI5DCCEASgCopWZVtbScphDDEB0BqKwoJrNgvSvLylkA5yilD70uz5umJISQAho1+6FOq+lhNIulC+YNEWpVKs3UrFBSH6TXQq1mIcsKzIKImFv3hVOX4qzx9xJ1PKd5JCvKbpNFnEcp/dvf3+N1CCEcgC65gvTDBNFandOoSbXyJaS61coa8ucOZjiNGgAgiDbcf5wsnbh43XQl7q6Wglo0LHsq1WiOBLCHUip7XDYfTVwPUUqbunE9AdAwWM9/apWkNp1b1KE92jfiq4eHIUjP273eJkmIuXUfOw6elZZs2m8DRUyq0TwDwDZKqdVVuTwBIaS4VqMeTggZUqVccTKke6ug+tXLoVC+ULvXUkpx5+FTH